In [1]:
import numpy as np
from random import shuffle
import os
# import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model
# # from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error, mean_absolute_error

# from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths to your image directories
# train_data_dir = 'D:\\UCD\\PhD_research_progress\\4-engagement_VA\\train_set\\images'
# validation_data_dir = 'D:\\UCD\\PhD_research_progress\\4-engagement_VA\\val_set\\images'
# test_data_dir = 'D:\\UCD\\PhD_research_progress\\4-engagement_VA\\train_set\\images'


In [2]:
import tensorflow as tf

C:\Users\rijju\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\framework\dtypes.py:511: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  np.object,


AttributeError: module 'numpy' has no attribute 'object'.
`np.object` was a deprecated alias for the builtin `object`. To avoid this error in existing code, use `object` by itself. Doing this will not modify any behavior and is safe. 
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [12]:
def image_generator(files, batch_size = 32, sz = (256, 256)):
  
  while True: 
    
    #extract a random batch 
    batch = np.random.choice(files, size = batch_size)    
    
    #variables for collecting batches of inputs and outputs 
    batch_x = []
    batch_y = []
    
    
    for f in batch:

        #get the masks. Note that masks are png files 
        mask = Image.open(f'annotations/trimaps/{f[:-4]}.png')
        mask = np.array(mask.resize(sz))


        #preprocess the mask 
        mask[mask >= 2] = 0 
        mask[mask != 0 ] = 1
        
        batch_y.append(mask)

        #preprocess the raw images 
        raw = Image.open(f'images/{f}')
        raw = raw.resize(sz)
        raw = np.array(raw)

        #check the number of channels because some of the images are RGBA or GRAY
        if len(raw.shape) == 2:
          raw = np.stack((raw,)*3, axis=-1)

        else:
          raw = raw[:,:,0:3]

        batch_x.append(raw)

    #preprocess a batch of images and masks 
    batch_x = np.array(batch_x)/255.
    batch_y = np.array(batch_y)
    batch_y = np.expand_dims(batch_y,3)

    yield (batch_x, batch_y)      

In [13]:
batch_size = 32

train_files = os.listdir('D:\\UCD\\PhD_research_progress\\4-engagement_VA\\train_set\\images')
shuffle(train_files)

_files = os.listdir('D:\\UCD\\PhD_research_progress\\4-engagement_VA\\val_set\\images')
split = int(0.95 * len(_files))

#split into training and testing
val_files = _files[0:split]
test_files  = _files[split:]

train_generator = image_generator(train_files, batch_size = batch_size)
val_generator  = image_generator(val_files, batch_size = batch_size)
test_generator  = image_generator(test_files, batch_size = batch_size)


In [15]:
train_generator

<generator object image_generator at 0x0000026956CCD4A0>

In [14]:


# Use the generators to load images and labels
X_train = train_generator.next()[0]
X_validation = val_generator.next()[0]
X_test = test_generator.next()[0]


# Split the data into training, validation, and test sets
# X_train, X_temp, y_valence_train, y_valence_temp, y_arousal_train, y_arousal_temp = train_test_split(X_images, y_valence, y_arousal, test_size=0.2, random_state=42)
# X_validation, X_test, y_valence_validation, y_valence_test, y_arousal_validation, y_arousal_test = train_test_split(X_temp, y_valence_temp, y_arousal_temp, test_size=0.5, random_state=42)



AttributeError: 'generator' object has no attribute 'next'

In [ ]:
# Define the CNN model architecture
image_input = Input(shape=(224, 224, 3))
x = Conv2D(32, (3, 3), activation='relu')(image_input)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)

# Create separate branches for valence and arousal predictions
valence_output = Dense(1, name='valence')(x)
arousal_output = Dense(1, name='arousal')(x)

# Create the model
combined_model = Model(inputs=image_input, outputs=[valence_output, arousal_output])

# Compile the model
combined_model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
combined_model.fit(X_train, [y_valence_train, y_arousal_train], epochs=10, batch_size=32, validation_data=(X_validation, [y_valence_validation, y_arousal_validation]))

# Evaluate the model
valence_pred, arousal_pred = combined_model.predict(X_test)
# valence_mse = mean_squared_error(y_valence_test, valence_pred)
# arousal_mse = mean_squared_error(y_arousal_test, arousal_pred)
# valence_mae = mean_absolute_error(y_valence_test, valence_pred)
# arousal_mae = mean_absolute_error(y_arousal_test, arousal_pred)

# print(f"Valence MSE: {valence_mse}, Arousal MSE: {arousal_mse}")
# print(f"Valence MAE: {valence_mae}, Arousal MAE: {arousal_mae}")


In [6]:
pip install --user tensorflow

Note: you may need to restart the kernel to use updated packages.
